In [1]:
from Bio import SeqIO
from glob import glob 
from subprocess import call, check_output
from itertools import product

In [6]:
# params and paths

read_dir = 'sanger_reads/'
my_sample_names = [ 'Alex{}-{}'.format( i, j ) for i, j in product( range( 1, 13 ), range( 1, 4 ) ) ]
wt_pep = 'order_peptide_sequences.fasta' 

In [7]:
# convert all the reads to FASTQ

for read in glob( '{}/*ab1'.format( read_dir ) ):
    SeqIO.convert( read, 'abi', read.replace( 'ab1', 'fastq' ), 'fastq' )

In [8]:
# pair and merge the reads 

expect = 0
for sample_name in my_sample_names:
    fwd = glob( '{}/*{}_*T7.fastq'.format( read_dir, sample_name ) ) 
    rev = glob( '{}/*{}_*T7-Ter.fastq'.format( read_dir, sample_name ) )
    if len( fwd ) == len( rev ) == 1:
        pear_opts = [ '-e', '2' ] 
        cmd = [ 'pear', '-f', fwd[0], '-r', rev[0], '-o', '{}/{}'.format( read_dir, sample_name ) ] + pear_opts
        call( cmd )
        expect += 1 

In [9]:
# convert all the assemblies to FASTA

for assembly in glob( '{}/*.assembled.fastq'.format( read_dir ) ):
    SeqIO.convert( assembly, 'fastq', assembly.replace( 'fastq', 'fasta' ), 'fasta' )

In [ ]:
! cat sanger_reads/*assembled.fasta > my_reads.fa 
! blastn -subject order_nucleotide_sequences.fasta -query my_reads.fa
! blastx -subject order_peptide_sequences.fasta -query my_reads.fa

In [ ]:
# diff the assemblies against the wild type protein sequence manually 

singles = []
for assembly in glob( '{}/*.assembled.fasta'.format( read_dir ) ):
    for wt_pep in screed.open( 'order_peptide_sequences.fasta' ): 
        
        name = assembly.split( '.' )[0] 
        
        with open( 'my_wt_pep', 'w' ) as fn:
            fn.write( '>{}\n{}\n'.format( name, wt_pep.sequence ) 
                     
        lines = check_output( [ 'blastx', '-subject', wt_pep, '-query', assembly, '-outfmt', '6 sseq qseq length' ] ).split()
                     
        if lines and lines[0] and lines[1] and lines[2]:

            z = zip( lines[0], lines[1] )
            d = [ '{}{}{}'.format( i, n+1, j ) for n, ( i, j ) in enumerate( z ) if i != j ]
            dstr = '+'.join( d ) 
            
            if len( d ) == 0: 
                print name, length, '\twild type'
            elif len( d ) > 1:
                print name, length, '\t{}'.format( dstr ) 
            else:
                print name, length, 'no alignment' 

In [48]:
# results 

